<a href="https://colab.research.google.com/github/hager104/medical_chatbot/blob/main/Copy_of_Untitled19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.3 MB/s eta 0:00:00


In [ ]:
!pip install ctransformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 47.0 MB/s eta 0:00:00


In [ ]:
!pip install Pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 7.4 MB/s eta 0:00:00


In [ ]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import CTransformers
from langchain.vectorstores import Pinecone
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
import pandas as panda

In [ ]:
PINECONE_API_key="pcsk_29YpbD_SHWVCkHpPQMnuto42uEqyduYCM2D3SeUNV5guDWeMNrj71rZMASqzNr26EvisMJ"
PINECONE_API_ENV="us-east-1"


In [ ]:
import pandas as pd
df = pd.read_csv("/content/Medical_Intelligence_Dataset (1).csv")
# Display the first 5 rows of the dataset
print(df.head())
# Display some basic information about the dataset
print(df.info())

                                               input  \
0            What is (are) atelosteogenesis type 1 ?   
1  :) hey i got a question .. so yeah i had a roo...   
2  How many people are affected by Urinary Tract ...   
3  hi thereI am 30 years old and a month ago i no...   
4  What are the symptoms of Spinocerebellar ataxi...   

                                              output  
0  Atelosteogenesis type 1 is a disorder that aff...  
1  Thanks for your query, I have gone through you...  
2  Urinary tract infections affect about 3 percen...  
3  Thanks for your question on Chat Doctor. I can...  
4  What are the signs and symptoms of Spinocerebe...  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40442 entries, 0 to 40441
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   input   40442 non-null  object
 1   output  40442 non-null  object
dtypes: object(2)
memory usage: 632.0+ KB
None


In [ ]:
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 1. clean the data
df = df.dropna(subset=['input'])
df = df[df['input'].str.strip() != '']

# processed function 'text_split'
def text_split(text):
    """division the data by RecursiveCharacterTextSplitter."""
    text_splitter = RecursiveCharacterTextSplitter(
       #Portion size
        chunk_size=500,
       #part overlap
        chunk_overlap=20

    )
    if isinstance(text, str) and pd.notna(text) and len(text) > 0:  # التحقق من أن النص ليس فارغًا
        return text_splitter.split_text(text)
    else:
        return [] # Added indentation here

#  implementation and create chunks
df['chunks'] = df['input'].apply(text_split)
df_chunked = df.explode('chunks', ignore_index=True)

print(df_chunked.head())


                                               input  \
0            What is (are) atelosteogenesis type 1 ?   
1  :) hey i got a question .. so yeah i had a roo...   
2  How many people are affected by Urinary Tract ...   
3  hi thereI am 30 years old and a month ago i no...   
4  hi thereI am 30 years old and a month ago i no...   

                                              output  \
0  Atelosteogenesis type 1 is a disorder that aff...   
1  Thanks for your query, I have gone through you...   
2  Urinary tract infections affect about 3 percen...   
3  Thanks for your question on Chat Doctor. I can...   
4  Thanks for your question on Chat Doctor. I can...   

                                              chunks  
0            What is (are) atelosteogenesis type 1 ?  
1  :) hey i got a question .. so yeah i had a roo...  
2  How many people are affected by Urinary Tract ...  
3  hi thereI am 30 years old and a month ago i no...  
4  had breast cancer on both breast years apart, ..

In [ ]:
print(f"length of chunks: {len(df_chunked)}")

length of chunks: 46925


In [ ]:
#downlioading hugging face embeddinds model
def download_hugging_face_embeddings():
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
  return embeddings

In [ ]:
embeddings=download_hugging_face_embeddings()

<ipython-input-12-f981c366a161>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or d

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [ ]:
query_result=embeddings.embed_query("hello world")
print("length=",len(query_result))


length= 384


In [ ]:
query_result

[-0.034477315843105316,
 0.031023172661662102,
 0.006734910886734724,
 0.02610892429947853,
 -0.03936195746064186,
 -0.1603025197982788,
 0.06692396104335785,
 -0.006441440898925066,
 -0.04745054617524147,
 0.014758836477994919,
 0.07087532430887222,
 0.055527545511722565,
 0.01919332519173622,
 -0.026251299306750298,
 -0.01010951679199934,
 -0.026940451934933662,
 0.022307397797703743,
 -0.022226639091968536,
 -0.1496926248073578,
 -0.01749303936958313,
 0.007676327601075172,
 0.054352276027202606,
 0.0032544792629778385,
 0.03172592446208,
 -0.08462144434452057,
 -0.029405953362584114,
 0.05159562826156616,
 0.048124104738235474,
 -0.003314818488433957,
 -0.05827919766306877,
 0.04196928068995476,
 0.02221069671213627,
 0.12818878889083862,
 -0.02233896404504776,
 -0.011656257323920727,
 0.06292840093374252,
 -0.03287629410624504,
 -0.09122602641582489,
 -0.031175386160612106,
 0.05269954726099968,
 0.047034841030836105,
 -0.08420310169458389,
 -0.030056146904826164,
 -0.020744822919

In [ ]:
import os
import pinecone
from langchain.vectorstores import Pinecone as LangchainPinecone

# Set Pinecone API key and environment as environment variables
os.environ["PINECONE_API_KEY"] = "pcsk_29YpbD_SHWVCkHpPQMnuto42uEqyduYCM2D3SeUNV5guDWeMNrj71rZMASqzNr26EvisMJ"  # Replace with your actual API key
os.environ["PINECONE_ENVIRONMENT"] = "us-east-1"

# Initialize Pinecone by creating an instance of the Pinecone class
pc = pinecone.Pinecone(
    api_key=os.environ["PINECONE_API_KEY"],
    environment=os.environ["PINECONE_ENVIRONMENT"]
)

# Define index name
index_name = "medicalchatbot"

# Create embeddings and store in Pinecone
docsearch = LangchainPinecone.from_texts(
    df_chunked['chunks'].tolist(),
    embeddings,
    index_name=index_name
)

In [ ]:

docsearch=LangchainPinecone.from_existing_index(index_name,embeddings)
query=" What is (are) atelosteogenesis type 1 ?"
docs=docsearch.similarity_search(query,k=3)
print("result",docs)

result [Document(metadata={}, page_content='What is (are) atelosteogenesis type 1 ?'), Document(metadata={}, page_content='What is (are) atelosteogenesis type 2 ?'), Document(metadata={}, page_content='What is (are) atelosteogenesis type 3 ?')]


In [ ]:
prompt_tamplate='''Use the following pieces of information to answer the user's question.
if you don't know the answer, just say that you don't know, don't try to make up an answer.
Context:{context}
Question:{question}

only return the helpful answer below and nothing else.
Helpful answer
 '''

In [ ]:
PROMPT=PromptTemplate(template=prompt_tamplate,input_variables=['context','question'])
chain_type_kwargs={"prompt":PROMPT}

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [ ]:
!pip install transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

llm = CTransformers(
    model="TheBloke/Llama-2-7B-Chat-GGML",  # Updated model path
    model_type="llama",
    config={
        'max_new_tokens': 512,
        'temperature': 0.8
    }
)


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

llama-2-7b-chat.ggmlv3.q2_K.bin:   0%|          | 0.00/2.87G [00:00<?, ?B/s]

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={"k":2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs,
)

In [ ]:
while True:
  user_input = input("input prompt:")
  if user_input.lower() == "exit":
    break
  result = qa({"query": user_input})
  print("response:", result['result'])

input prompt:hey i got a question .. so yeah i had a root canal on one tooth and being lazy i forgot to get the permanent filling and as time passed my tooth fell out .. so my question is .. Can i still get infection there even if i had a root canal and forgot to get the filling and my tooth fell out ??
response:  I understand your concern, but I'm afraid you can't have sex with your girlfriend until her tooth infection has been fully treated and is no longer contagious. The infection from an abscessed tooth can spread to other parts of the body through the bloodstream, including the heart, brain, or lungs, which could lead to serious complications. It's important to prioritize your girlfriend's health and safety above all else. Once she has completed her antibiotic treatment and has received clearance from her doctor, it will be safe for you to engage in sexual activities with each other again.
input prompt:exit
